## Notebook to create a rainfall-runoff model using data driven methods

**General Description**

The following notebook contains the code to create, train, validate, and test a rainfall-runoff model using a Hybrid Hydrological approach. We use an LSTM network to parameterize a process based rainfall-runoff model. The code allows for the creation of single-basin models, but it is conceptualized to create regional models. The code is intended as an intial introduction to the topic, 
in which we prioritized interpretability over modularity.

The logic of the code is heavily based on [Neural Hydrology](https://doi.org/10.21105/joss.04050) [1]. For a more flexible, robust, and modular implementation of deep learning methods in hydrological modeling, we advise the use of Neural Hydrology. 

**Experiment Details**
- In this example we use the Hybrid model architecture, to create a regional rainfall-runoff model for 531 basins of the CAMELS_US dataset [2], [3].

**Authors:**
- Eduardo Acuña Espinoza (eduardo.espinoza@kit.edu)
- Ralf Loritz
- Manuel Álvarez Chaves

**References:**

[1]: Kratzert, F., Gauch, M., Nearing, G., & Klotz, D. (2022). NeuralHydrology – A Python library for deep learning research in hydrology. Journal of Open Source Software, 7, 4050. https://doi.org/10.21105/joss.04050

[2]: Newman, A. J., Clark, M. P., Sampson, K., Wood, A., Hay, L. E., Bock, A., Viger, R. J., Blodgett, D., Brekke, L., Arnold, J. R., Hopson, T., & Duan, Q. (2015). Development of a large-sample watershed-scale hydrometeorological dataset for the contiguous USA: dataset characteristics and assessment of regional variability in hydrologic model performance. *Hydrology and Earth System Sciences, 19*, 209-223. https://doi.org/10.5194/hess-19-209-2015

[3]: Addor, N., Newman, A. J., Mizukami, N., & Clark, M. P. (2017). The CAMELS data set: catchment attributes and meteorology for large-sample studies. *Hydrology and Earth System Sciences, 21*, 5293-5313. https://doi.org/10.5194/hess-21-5293-2017

In [1]:
# Import necessary packages
import pickle
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader

sys.path.append("..")
# Import classes and functions from other files
from hy2dl.aux_functions.functions_evaluation import nse
from hy2dl.aux_functions.functions_training import nse_basin_averaged, weighted_rmse
from hy2dl.aux_functions.utils import create_folder, set_random_seed, upload_to_device, write_report
from hy2dl.datasetzoo.camelsus import CAMELS_US as Datasetclass
from hy2dl.modelzoo.hbv import HBV as conceptual_model
from hy2dl.modelzoo.hybrid import Hybrid as modelclass
from hy2dl.modelzoo.uh_routing import UH_routing as routing_model

Part 1. Initialize information

In [2]:
# Define experiment nae
experiment_name = "Hybrid_CAMELS_US"

# paths to access the information
path_entities = "../../data/basin_id/basins_camels_us_3.txt"
path_data = "../../data/CAMELS_US"
path_additional_features = "../../data/CAMELS_US/pet_hargreaves.pickle"

# dynamic forcings and target
dynamic_input = ["prcp(mm/day)", "srad(W/m2)", "tmax(C)", "tmin(C)", "vp(Pa)", "dayl(s)"]
conceptual_input = ["prcp(mm/day)", "pet(mm/day)", "tmax(C)", "tmin(C)"]
forcing = ["daymet"]
target = ["QObs(mm/d)"]

# static attributes that will be used. If one is not using static_inputs, initialize the variable as an empty list.
static_input = [
    "p_mean",
    "pet_mean",
    "p_seasonality",
    "frac_snow",
    "aridity",
    "high_prec_freq",
    "high_prec_dur",
    "low_prec_freq",
    "low_prec_dur",
    "elev_mean",
    "slope_mean",
    "area_gages2",
    "frac_forest",
    "lai_max",
    "lai_diff",
    "gvf_max",
    "gvf_diff",
    "dom_land_cover_frac",
    "dom_land_cover",
    "root_depth_50",
    "soil_depth_pelletier",
    "soil_depth_statsgo",
    "soil_porosity",
    "soil_conductivity",
    "max_water_content",
    "sand_frac",
    "silt_frac",
    "clay_frac",
    "geol_1st_class",
    "glim_1st_class_frac",
    "geol_2nd_class",
    "glim_2nd_class_frac",
    "carbonate_rocks_frac",
    "geol_porostiy",
    "geol_permeability",
]

# time periods
training_period = ["1980-10-01", "1995-09-30"]
validation_period = ["1980-12-31", "1985-09-30"]
testing_period = ["1995-10-01", "2010-09-30"]


# model configuration
model_configuration = {
    "input_size_lstm": len(dynamic_input) + len(static_input),
    "no_of_layers": 1,
    "seq_length": 730,
    "predict_last_n": 365,
    "unique_prediction_blocks": False,
    "hidden_size": 256,
    "conceptual_model": conceptual_model,
    "routing_model": routing_model,
    "batch_size_training": 256,
    "batch_size_evaluation": 100,  # number of basins that will be run for the whole period
    "no_of_epochs": 3,
    "max_updates_per_epoch": 450,
    "learning_rate": 0.001,
    "set_forget_gate": 3,
    "n_conceptual_models": 16,
    "conceptual_dynamic_parameterization": ["BETA", "BETAET"],
    "validate_every": 1,
}

# device to train the model
running_device = "gpu"  # cpu or gpu

# define random seed
seed = 110

# colorblind friendly palette
color_palette = {"observed": "#377eb8","simulated": "#4daf4a"}

Part 2. Calculate additional information necessary to run the model

In [ ]:
# Create folder to store the results
path_save_folder = "../results/" + experiment_name + "_seed_" + str(seed)
create_folder(folder_path=path_save_folder)

In [ ]:
# check if model will be run in gpu or cpu and define device
if running_device == "gpu":
    print(torch.cuda.get_device_name(0))
    device = "cuda:0"
elif running_device == "cpu":
    device = "cpu"

Part 3. Class to create the dataset object used in training

In [5]:
# Dataset training
training_dataset = Datasetclass(
    dynamic_input=dynamic_input,
    forcing=forcing,
    target=target,
    sequence_length=model_configuration["seq_length"],
    time_period=training_period,
    path_data=path_data,
    path_entities=path_entities,
    path_additional_features=path_additional_features,
    check_NaN=True,
    predict_last_n=model_configuration["predict_last_n"],
    static_input=static_input,
    conceptual_input=conceptual_input,
    unique_prediction_blocks=model_configuration["unique_prediction_blocks"],
)

training_dataset.calculate_basin_std()
training_dataset.calculate_global_statistics(path_save_scaler=path_save_folder)
training_dataset.standardize_data(standardize_output=False)

In [ ]:
# Dataloader training
train_loader = DataLoader(
    dataset=training_dataset,
    batch_size=model_configuration["batch_size_training"],
    shuffle=True,
    drop_last=False,
    collate_fn=training_dataset.collate_fn,
)

# Print details of a loader´s sample to see that our format is correct
print("Number of batches in training: ", len(train_loader))
print("\nSample batch details:")
print(f"{'Key':<12} | {'Shape':<20}")
print("-" * 35)
# Loop through the sample dictionary and print the shape of each element
for key, value in next(iter(train_loader)).items():
    print(f"{key:<12} | {str(value.shape):<20}")

Part 4. Create dataset for validation

In [ ]:
# When we run our evalation period (validation or testing), we want to differentiate between basins. Therefore, each 
# batch entity will contain the whole time period (validation or testing) of a specific basin. For this, we will modify 
# seq_length and the predict_last_n.
warmup_start_date = pd.to_datetime(validation_period[0], format="%Y-%m-%d") - pd.DateOffset(
    model_configuration["seq_length"] - model_configuration["predict_last_n"]
)
evaluation_seq_length = (pd.to_datetime(validation_period[1], format="%Y-%m-%d") - warmup_start_date).days + 1
evaluation_predict_last_n = evaluation_seq_length - model_configuration["predict_last_n"]

validation_dataset = Datasetclass(
    dynamic_input=dynamic_input,
    forcing=forcing,
    target=target,
    sequence_length=evaluation_seq_length,
    time_period=validation_period,
    path_data=path_data,
    path_entities=path_entities,
    path_additional_features=path_additional_features,
    check_NaN=False,
    predict_last_n=evaluation_predict_last_n,
    static_input=static_input,
    conceptual_input=conceptual_input,
)

validation_dataset.scaler = training_dataset.scaler  # read the global statistics calculated in the training period
validation_dataset.standardize_data(standardize_output=False)

# DataLoader testing
validation_loader = DataLoader(
    validation_dataset,
    batch_size=model_configuration["batch_size_training"],
    shuffle=False,
    drop_last=False,
    collate_fn=training_dataset.collate_fn,
)

# Print details of a loader´s sample to see that our format is correct
print("Number of batches in training: ", len(validation_loader))
print("\nSample batch details:")
print(f"{'Key':<12} | {'Shape':<20}")
print("-" * 35)
# Loop through the sample dictionary and print the shape of each element
for key, value in next(iter(validation_loader)).items():
    print(f"{key:<12} | {str(value.shape):<20}")

Part 5. Train Model

In [ ]:
# construct model
set_random_seed(seed=seed)
model = modelclass(model_configuration=model_configuration).to(device)

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=model_configuration["learning_rate"])

# set forget gate to 3 to ensure that the model is capable to learn long term dependencies
model.lstm.bias_hh_l0.data[model_configuration["hidden_size"] : 2 * model_configuration["hidden_size"]] = (
    model_configuration["set_forget_gate"]
)

training_time = time.time()
# Loop through the different epochs
for epoch in range(1, model_configuration["no_of_epochs"] + 1):
    epoch_start_time = time.time()
    total_loss = []
    # Training -------------------------------------------------------------------------------------------------------
    model.train()
    for idx, sample in enumerate(train_loader):
        # maximum iterations per epoch
        if (
            model_configuration.get("max_updates_per_epoch") is not None
            and idx >= model_configuration["max_updates_per_epoch"]
        ):
            break

        sample = upload_to_device(sample, device)  # upload tensors to device
        optimizer.zero_grad()  # sets gradients of weigths and bias to zero

        pred = model(sample)  # forward call

        loss = nse_basin_averaged(y_sim=pred["y_hat"], y_obs=sample["y_obs"], per_basin_target_std=sample["basin_std"])

        loss.backward()  # backpropagates
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)  # clip gradients
        optimizer.step()  # update weights
        total_loss.append(loss.item())

        # remove from cuda
        del sample, pred
        torch.cuda.empty_cache()

    # training report
    report = f'Epoch: {epoch:<2} | Loss training: {"%.3f "% (np.mean(total_loss))}'

    # Validation -----------------------------------------------------------------------------------------------------
    if epoch % model_configuration["validate_every"] == 0:
        model.eval()
        with torch.no_grad():
            validation_results = {}
            for sample in validation_loader:
                sample = upload_to_device(sample, device)  # upload tensors to device
                pred = model(sample)  # forward call # forward call

                # join results in a dataframe and store them in a dictionary (is easier to plot later)
                for i in range(pred["y_hat"].shape[0]):
                    df_discharge = pd.DataFrame(
                        data={
                            "y_obs": sample["y_obs"][i, :, 0].flatten().cpu().detach().numpy(),
                            "y_sim": pred["y_hat"][i, :, 0].flatten().cpu().detach().numpy(),
                        },
                        index=pd.to_datetime(sample["date"][i, :].flatten()),
                    )

                    validation_results[sample["basin"][i]] = df_discharge

                # remove from cuda
                del sample, pred
                torch.cuda.empty_cache()

        # average loss validation
        loss_validation = nse(df_results=validation_results)
        report += f'| NSE validation: {"%.3f "% (loss_validation)}'

    # save model after every epoch
    path_saved_model = path_save_folder + "/epoch_" + str(epoch)
    torch.save(model.state_dict(), path_saved_model)

    # print epoch report
    report += (
        f'| Epoch time: {"%.1f "% (time.time()-epoch_start_time)} s | LR:{"%.5f "% (optimizer.param_groups[0]["lr"])}'
    )
    print(report)
    write_report(file_path=path_save_folder + "/run_progress.txt", text=report)

# print final report
report = f'Total training time: {"%.1f "% (time.time()-training_time)} s'
print(report)
write_report(file_path=path_save_folder + "/run_progress.txt", text=report)

Part 6. Test LSTM

In [9]:
# In case I already trained an LSTM I can re-construct the model
# model = modelclass(model_configuration=model_configuration).to(device)
# model.load_state_dict(torch.load(path_save_folder + "/epoch_20", map_location=device))

# We can read the training scaler or read a previously stored one
scaler = training_dataset.scaler
# with open(path_save_folder + "/scaler.pickle", "rb") as file:
#    scaler = pickle.load(file)

In [ ]:
# When we run our evalation period (validation or testing), we want to differentiate between basins. Therefore, each 
# batch entity will contain the whole time period (validation or testing) of a specific basin. For this, we will modify 
# seq_length and the predict_last_n.
warmup_start_date = pd.to_datetime(testing_period[0], format="%Y-%m-%d") - pd.DateOffset(
    model_configuration["seq_length"] - model_configuration["predict_last_n"]
)
evaluation_seq_length = (pd.to_datetime(testing_period[1], format="%Y-%m-%d") - warmup_start_date).days + 1
evaluation_predict_last_n = evaluation_seq_length - model_configuration["predict_last_n"]

testing_dataset = Datasetclass(
    dynamic_input=dynamic_input,
    forcing=forcing,
    target=target,
    sequence_length=evaluation_seq_length,
    time_period=testing_period,
    path_data=path_data,
    path_entities=path_entities,
    path_additional_features=path_additional_features,
    check_NaN=False,
    predict_last_n=evaluation_predict_last_n,
    static_input=static_input,
    conceptual_input=conceptual_input,
)

testing_dataset.scaler = scaler  # read the global statistics calculated in the training period
testing_dataset.standardize_data(standardize_output=False)

# DataLoader testing
test_loader = DataLoader(
    testing_dataset,
    batch_size=model_configuration["batch_size_training"],
    shuffle=False,
    drop_last=False,
    collate_fn=training_dataset.collate_fn,
)

# Print details of a loader´s sample to see that our format is correct
print("Number of batches in training: ", len(test_loader))
print("\nSample batch details:")
print(f"{'Key':<12} | {'Shape':<20}")
print("-" * 35)
# Loop through the sample dictionary and print the shape of each element
for key, value in next(iter(test_loader)).items():
    print(f"{key:<12} | {str(value.shape):<20}")

In [11]:
# Testing ----------------------------------------------------------------------
model.eval()
with torch.no_grad():
    test_results = {}
    for sample in test_loader:
        sample = upload_to_device(sample, device)  # upload tensors to device
        pred = model(sample)  # forward call

        # join results in a dataframe and store them in a dictionary (is easier to plot later)
        for i in range(pred["y_hat"].shape[0]):
            df_discharge = pd.DataFrame(
                data={
                    "y_obs": sample["y_obs"][i, :, 0].flatten().cpu().detach().numpy(),
                    "y_sim": pred["y_hat"][i, :, 0].flatten().cpu().detach().numpy(),
                },
                index=pd.to_datetime(sample["date"][i, :].flatten()),
            )

            # extract internal_state (buckets) information
            internal_states = {
                key: value[i, :, :].squeeze(0).cpu().detach().numpy() for key, value in pred["internal_states"].items()
            }

            # extract parameter  information
            parameters = {
                key: value[i, :, :].squeeze(0).cpu().detach().numpy() for key, value in pred["parameters"].items()
            }

            test_results[sample["basin"][i]] = {
                "discharges": df_discharge,
                "internal_states": internal_states,
                "parameters": parameters,
            }

        # remove from cuda
        del sample, pred
        torch.cuda.empty_cache()

# Save results as a pickle file
# with open(path_save_folder+"/test_results.pickle", "wb") as f:
#   pickle.dump(test_results, f)

Part 7. Initial analysis

In [12]:
discharges = {key: value["discharges"] for key, value in test_results.items()}
loss_testing = nse(df_results=discharges, average=False)
df_NSE = pd.DataFrame(data={"basin_id": list(test_results.keys()), "NSE": np.round(loss_testing, 3)})
df_NSE = df_NSE.set_index("basin_id")
df_NSE.to_csv(path_save_folder + "/NSE.csv", index=True, header=True)

In [ ]:
# Plot the histogram
plt.hist(df_NSE["NSE"], bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

# Add NSE statistics to the plot
plt.text(
    0.01,
    0.8,
    (
        f'Mean: {"%.2f" % df_NSE["NSE"].mean():>7}\n'
        f'Median: {"%.2f" % df_NSE["NSE"].median():>0}\n'
        f'Max: {"%.2f" % df_NSE["NSE"].max():>9}\n'
        f'Min: {"%.2f" % df_NSE["NSE"].min():>10}'
    ),
    transform=plt.gca().transAxes,
    bbox=dict(facecolor="white", alpha=0.5),
)

# Format plot
plt.rcParams["figure.figsize"] = (20, 5)
plt.xlabel("NSE", fontsize=12, fontweight="bold")
plt.ylabel("Frequency", fontsize=12, fontweight="bold")
plt.title("NSE Histogram", fontsize=16, fontweight="bold")
# plt.savefig(save_folder+"/NSE_Histogram.png", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
# Plot simulated and observed discharges
basin_to_analyze = "01022500"
plt.plot(test_results[basin_to_analyze]["discharges"]["y_obs"], label="observed", color=color_palette["observed"])
plt.plot(
    test_results[basin_to_analyze]["discharges"]["y_sim"],
    label="simulated",
    alpha=0.5,
    color=color_palette["simulated"],
)

# Format plot
plt.xlabel("Day", fontsize=12, fontweight="bold")
plt.ylabel("Discharge [mm/d]", fontsize=12, fontweight="bold")
plt.title("Discharge comparison", fontsize=16, fontweight="bold")
plt.tick_params(axis="both", which="major", labelsize=12)
plt.legend(loc="upper right", fontsize=12)
# plt.savefig(save_folder+"/Model_Comparison.png", bbox_inches="tight", pad_inches=0)

In [ ]:
# Plot states
state_of_interest = "SM"

states = test_results[basin_to_analyze]["internal_states"][state_of_interest]

for i in range(states.shape[1]):
    plt.plot(
        test_results[basin_to_analyze]["discharges"]["y_obs"].index,
        states[:, i],
        label=state_of_interest + "_" + str(i + 1),
    )

# Adding labels and legend
plt.xlabel("Day", fontsize=12, fontweight="bold")
plt.ylabel("State [mm/d]", fontsize=12, fontweight="bold")
plt.title("Time evolution of internal states of conceptual model", fontsize=16, fontweight="bold")
plt.tick_params(axis="both", which="major", labelsize=12)
plt.legend(loc="upper right", fontsize=12)
plt.show()

In [ ]:
# Plot states
parameter_of_interest = "BETA"

states = test_results[basin_to_analyze]["parameters"][parameter_of_interest]

for i in range(states.shape[1]):
    plt.plot(
        test_results[basin_to_analyze]["discharges"]["y_obs"].index,
        states[:, i],
        label=parameter_of_interest + "_" + str(i + 1),
        alpha=0.8,
    )

# Adding labels and legend
plt.xlabel("Day", fontsize=12, fontweight="bold")
plt.ylabel("State [mm/d]", fontsize=12, fontweight="bold")
plt.title("Time evolution of parameters of conceptual model", fontsize=16, fontweight="bold")
plt.tick_params(axis="both", which="major", labelsize=12)
plt.legend(loc="upper right", fontsize=12)
plt.show()